## IO example for  phenol atomic positions optimization (in a box) using Conquest
> The objective is to use ASE for pre-process (building the **I**nput files) and
> post-process (having the **O**utput file) a Conquest calculation. 
> The calculation *must* be run at hand,
> that is, the calculation is not launched by ASE.

<div class="alert alert-block alert-danger">
    <b>WARNING:</b> IO example requirements
</div>    

> `ASE version = 3.20.0b1` including `Conquest`


In [1]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt

from distutils.spawn import find_executable
from numpy import amax, amin, matmul, exp, pi, array, zeros, sqrt
from scipy.constants import Rydberg

from ase.build import bulk
from ase.calculators.conquest import Conquest
from ase.io.conquest_f_rework_output_ase import read_conquest, read_conquest_out
from ase.io.conquest_f_rework_output_ase import Conquest_orthorhombic_check
from ase.units import Rydberg

from ase.dft.bandgap import bandgap
from ase.io import read

from ase import Atoms
from ase.spacegroup import crystal

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import print_occ, get_gapwind
from cq_ase_external_lib import cq_postproc_xsf, cq_print_output_data


In [2]:
%%bash 
ase --version

ase-3.20.0b1_cq


#### Check if visualisation tools are installed

In [4]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

xcrysden found
VESTA found
VMDLauncher not found


#### Define Conquest environment

In [5]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-release-f-rework-output-ase/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/Conquest-develop-outdated/pseudo-and-pao'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-release-develop/tools/BasisGeneration/MakeIonFiles'

#### Directory for storing calculation files

In [6]:
working_directory = 'cq_example_phenol_optpos_io'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

####  Build ASE `Atoms` for C$_6$H$_{5}$OH in a box

- Load structure of C$_6$H$_{5}$OH from `PDB` file 

In [7]:
struct = read('./struct/phenol.pdb',0)

print('n atoms = {} ?'.format(len(struct)))

n atoms = 13 ?


- Cell parameters in Ang. and deg. [a, b, c, alpha, beta, gamma]. Cell size should be cubic $8\times8\times8$ Ang.

In [8]:
a = 10.0
b = a
c = a
alpha = 90.0
beta  = 90.0
gamma = 90.0

cell = [a,b,c,alpha,beta,gamma]


- Generate the `Atoms`
 object and store in `struct` 

In [9]:
struct = Atoms(struct,cell=cell,pbc=[True,True,True])

# Conquest can only handle orthorhombic cells
struct = Conquest_orthorhombic_check(struct,verbose=True)

- Extract and print main structural data from input structure `struct`

In [10]:
print_struct_data(struct,verbose=1)

Cartesian atomic positions (Ang.)
  C       0.000000      0.000000      0.000000
  C       0.000000      0.000000      1.400000
  C       1.212000      0.000000      2.100000
  C       2.425000      0.000000      1.400000
  C       2.425000      0.000000      0.000000
  C       1.212000      0.000000     -0.700000
  H      -0.943000      0.000000      1.944000
  H       1.212000      0.000000      3.189000
  H       3.368000      0.000000      1.944000
  H       3.368000      0.000000     -0.545000
  H       1.212000      0.000000     -1.789000
  O      -1.224000     -0.021000     -0.679000
  H      -1.067000     -0.018000     -1.615000

Fractional atomic positions (Adim.)
  C       0.000000      0.000000      0.000000
  C       0.000000      0.000000      0.140000
  C       0.121200      0.000000      0.210000
  C       0.242500      0.000000      0.140000
  C       0.242500      0.000000      0.000000
  C       0.121200      0.000000     -0.070000
  H      -0.094300      0.000000    

- Save input `struct` in `CIF` and `XSF` format 

In [11]:
struct.write(working_directory+'/input.xsf')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input structure (if possible)

In [12]:
if ( 'VESTA' in cmd_app ):
    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

#### Setup Conquest atomic basis set

In [13]:
basis = {
         'O' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },
         'C' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },         
          }

#basis = {
#         'O' : { 'file' : 'O.ion', 'pseudopotential_type' : 'siesta'},             
#         'C' : { 'file' : 'C.ion', 'pseudopotential_type' : 'siesta'},             
#         'H' : { 'file' : 'H.ion', 'pseudopotential_type' : 'siesta'}             
#}

#### Setup calculation using Conquest as calculator

In [14]:
cutoff  =  90.0
kpoints = [1,1,1]
fxc     = 'PBE'

conquest_flags = {'IO.WriteOutToASEFile': True}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'sqnm',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  5e-3,   # max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       }) 

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

struct.calc = calc

input basis:
O {'gen_basis': True, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
input basis:
H {'gen_basis': True, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
input basis:
C {'gen_basis': True, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}


#### Generate Conquest input and write

- input file `Conquest_input` is stored in `./working_directory`

In [15]:
struct.calc.write_input(struct)

## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/calculators/conquest.py , line: 275
>>>>  H.ion taken from cq_example_phenol_optpos_io/H.ion
## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/calculators/conquest.py , line: 275
>>>>  C.ion taken from cq_example_phenol_optpos_io/C.ion
## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/calculators/conquest.py , line: 275
>>>>  O.ion taken from cq_example_phenol_optpos_io/O.ion
output basis:
H {'gen_basis': True, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
output basis:
C {'gen_basis': True, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
output basis:
O {'gen_basis': True, 'basis_size': 'small', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}


<div class="alert alert-block alert-danger">
    <b>WARNING:</b> you must have run the calculation somewhere and store the output files in the working directory
    </b>
</div>
Mandatory output file(s):

- `Conquest_out_ase`

Optional for struture analysis:
- `trajectory.xsf`
- `coord_next.dat`

#### Post-process `trajectory.xsf` file generated by Conquest

In [16]:
traj_file= 'trajectory.xsf'
traj_file=  working_directory+'/'+traj_file

if ( os.path.isfile(traj_file) ):
    # cq_postproc_xsf output last structure from xsf file
    struct_last = cq_postproc_xsf(traj_file,len(struct))
else:
    print('WARNING: {} not found !'.format(traj_file))

- or extract last structure from Conquest file `coord_next.dat` 

In [25]:
last_struct='coord_next.dat'
last_struct=working_directory+'/'+last_struct

if( os.path.isfile(last_struct) ):

    struct_last = read_conquest(last_struct, fractional=True, atomic_order=['H','C','O'])
    
    struct_last.write(working_directory+'/output_.cif')
    struct_last.write(working_directory+'/output_.xsf')
    
else:
    print('WARNING: {} not found !'.format(traj_file))

- Save ouput `struct_last` in `CIF` and `XSF` format 

In [26]:
struct_last.write(working_directory+'/output.cif')
struct_last.write(working_directory+'/output.xsf')

- run `xcrysden` to visualize the optimization movie (if possible)

In [27]:
if ( cmd_vis['xcrysden'] and os.path.isfile(traj_file) ):
    subprocess.run(['xcrysden', '--axsf', traj_file])


+-----------------------------------------------------------------+
|*****************************************************************|
|*                                                               *|
|*  XCrySDen -- (X-Window) CRYstalline Structures and DENsities  *|
|*               =         ===         =              ===        *|
|*---------------------------------------------------------------*|
|*                                                               *|
|*    Anton Kokalj (tone.kokalj@ijs.si)                          *|
|*    Jozef Stefan Institute, Ljubljana, Slovenia                *|
|*                                                               *|
|*    Copyright (c) 1996--2019 by Anton Kokalj                   *|
|*                                                               *|
|*****************************************************************|
+-----------------------------------------------------------------+

  Version: 1.6.2

  Please report bugs to: ton

Running on platform : unix
   Operating system : Darwin
Package Xwd: /opt/X11/bin/xwd
Executing: /opt/local/lib/xcrysden-1.6.2/ftnunit
Viewer: mesaWid = .mesa
*** xsfAnimInit /var/folders/l5/s99fcqc90556khv0qm30stxh0000gr/T//xc_18793/trajectory.xsf.raw .mesa
reading PRIMCOORD
Number of Atoms:  0
Number of Frames: 0
Estimated number of bonds = 0
sInfo(dim) = 3; periodic(dim) = 0
Executing: /opt/local/lib/xcrysden-1.6.2/gengeom 0 1 11 1 1 1 1 xc_gengeom.18793 /var/folders/l5/s99fcqc90556khv0qm30stxh0000gr/T//xc_18793/trajectory.xsf.raw.1.raw
SetWatchCursor: .menu
SetWatchCursor: .mesa
SetWatchCursor: .ctrl
SetWatchCursor: .mea
SetWatchCursor: .close
SetWatchCursor: .#BWidget
Executing: /opt/local/lib/xcrysden-1.6.2/gengeom 2 1 11 1 1 1 1 /var/folders/l5/s99fcqc90556khv0qm30stxh0000gr/T//xc_18793/xc_struc.18793 /var/folders/l5/s99fcqc90556khv0qm30stxh0000gr/T//xc_18793/trajectory.xsf.raw.1.raw
SetWatchCursor: .menu
SetWatchCursor: .mesa
SetWatchCursor: .ctrl
SetWatchCursor: .mea
SetWatchC

#### Read Conquest output results and print the data

In [28]:
struct.calc.read_results(struct)

cq_print_output_data(struct,2)

Total electronic energy =   -1517.267468 eV
Atoms kinetic energy    =       0.000000 eV
Total energy =   -1517.267468 eV

Atomic forces (eV/Ang)
  C    0.039875   -0.015450    0.074775
  C   -0.013834   -0.007161    0.140791
  C   -0.024067    0.003878    0.017798
  C    0.026644    0.003180    0.023146
  C   -0.035339    0.003774   -0.104298
  C   -0.030841   -0.008137    0.027217
  H   -0.053174   -0.003648    0.044576
  H   -0.018919    0.003569    0.008939
  H    0.001919   -0.001289   -0.027428
  H   -0.026812    0.003641   -0.028046
  H   -0.004671   -0.003498   -0.039866
  O    0.129597    0.021842   -0.105163
  H   -0.012914   -0.003087   -0.060001

Cell stress tensor components (eV/Ang**3)
  xx =    0.000344
  yy =   -0.000029
  zz =   -0.000528
  yz =    0.000000
  xz =    0.000000


#### Extract and print main structural data from optimized structure `last_struct`

In [33]:
print_struct_data(struct_last,verbose=2)

Cartesian atomic positions (Ang.)
  C       9.955080      9.995827      9.978130
  C       9.969211      0.000521      1.420770
  C       1.212958      0.000910      2.123587
  C       2.457394      9.999959      1.415840
  C       2.444533      0.000986      9.985170
  C       1.205205      0.000469      9.261664
  H       8.969736      9.999459      1.995586
  H       1.208240      0.000129      3.275849
  H       3.454185      9.999828      1.992899
  H       3.443727      0.000120      9.408341
  H       1.214537      9.999467      8.110705
  O       8.703846      9.979446      9.342644
  H       8.962939      9.982321      8.338779

Fractional atomic positions (Adim.)
  C       0.995508      0.999583      0.997813
  C       0.996921      0.000052      0.142077
  C       0.121296      0.000091      0.212359
  C       0.245739      0.999996      0.141584
  C       0.244453      0.000099      0.998517
  C       0.120521      0.000047      0.926166
  H       0.896974      0.999946    

#### Extract Fermi energy, band gap and energy levels

In [34]:
# Fermi energy as evaluated in Conquest code 'Fermi energy for spin'
fermi_energy = calc.get_fermi_level()
eigenvalues  = calc.get_eigenvalues()

print('Total electronic energy = {:14.6f} eV'.format(struct.calc.results['energy']))
print('Fermi energy            = {:14.6f} eV'.format(fermi_energy))
print_occ(eigenvalues,fermi_energy,0.01,method='aufbau',label='G')

# Evaluate band gap
gap, p1, p2  = bandgap(struct.calc,efermi=fermi_energy,direct=True)

Total electronic energy =   -1517.267468 eV
Fermi energy            =      -3.429805 eV

Band energies at the G point
# energy in eV (occupation number)
      82.72511 (0.000)
      79.02133 (0.000)
      77.64300 (0.000)
      72.84193 (0.000)
      67.98475 (0.000)
      67.03068 (0.000)
      65.14112 (0.000)
      52.42281 (0.000)
      50.67533 (0.000)
      47.82931 (0.000)
      47.78918 (0.000)
      44.56539 (0.000)
      44.14880 (0.000)
      42.17595 (0.000)
      40.15447 (0.000)
      39.42287 (0.000)
      38.17263 (0.000)
      37.34455 (0.000)
      36.57399 (0.000)
      31.88977 (0.000)
      31.71561 (0.000)
      30.98454 (0.000)
      30.79332 (0.000)
      28.77877 (0.000)
      27.93121 (0.000)
      27.42718 (0.000)
      25.40100 (0.000)
      24.78139 (0.000)
      24.13028 (0.000)
      23.55028 (0.000)
      22.85516 (0.000)
      22.80339 (0.000)
      20.25539 (0.000)
      20.02643 (0.000)
      19.04182 (0.000)
      18.21275 (0.000)
      17.20187 (0.0